1. Data Import and Preprocessing
In this section, we load the provided dataset and perform initial preprocessing. We combine the date and time columns into a single timestamp, sort the data by parking lot and time, and prepare it for further analysis. This step ensures that all subsequent feature engineering and modeling are based on clean, chronologically ordered data.

In [3]:
# Install required packages
!pip install bokeh pathway --quiet

import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

output_notebook()

# Load the dataset
df = pd.read_csv('dataset.csv')

# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Feature Engineering
We create new features to capture the dynamics of parking demand. This includes calculating the occupancy rate, mapping vehicle types to weights (to reflect willingness to pay), and converting traffic conditions into numeric values. These engineered features will be used in our demand and pricing models.

In [4]:
# Calculate occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Map vehicle types to weights
vehicle_type_weights = {'truck': 1.2, 'car': 1.0, 'bike': 0.7, 'cycle': 0.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights)

# Map traffic conditions to numeric values
traffic_map = {'low': 0, 'average': 0.5, 'high': 1}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

3. Demand Function Construction
We define a demand function that integrates multiple features: occupancy rate, queue length, traffic level, special day indicator, and vehicle type weight. The raw demand score is then normalized within each parking lot to ensure stable and comparable pricing adjustments across different lots.

In [5]:
# Normalize function for demand
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-6)

# Calculate raw and normalized demand
df['DemandRaw'] = (
    1.0 * df['OccupancyRate'] +
    0.5 * df['QueueLength'] -
    0.3 * df['TrafficLevel'] +
    0.7 * df['IsSpecialDay'] +
    0.4 * df['VehicleTypeWeight']
)
df['DemandNorm'] = df.groupby('SystemCodeNumber')['DemandRaw'].transform(normalize)

4. Pricing Models
We implement two core pricing models:

Baseline Linear Model: Price increases linearly with occupancy rate.

Demand-Based Model: Price is adjusted based on the normalized demand score.

Both models are bounded to ensure prices remain within a reasonable range. These models provide a foundation for dynamic, data-driven pricing.

In [6]:
base_price = 10
lambda_ = 0.8

# Demand-based pricing
def price_demand(row):
    price = base_price * (1 + lambda_ * row['DemandNorm'])
    return np.clip(price, 0.5 * base_price, 2 * base_price)

df['Price'] = df.apply(price_demand, axis=1)

# Baseline linear model
alpha = 2.0
df['PriceLinear'] = base_price + alpha * df['OccupancyRate']
df['PriceLinear'] = np.clip(df['PriceLinear'], 0.5 * base_price, 2 * base_price)

5. Real-Time Pricing Visualization
This section visualizes the evolution of parking prices over time for each lot. The plots compare the demand-based price (solid line) and the baseline linear price (dashed line), helping to illustrate the impact of different pricing strategies in real time.

In [7]:
plots = []
for lot in df['SystemCodeNumber'].unique()[:4]:  # Show first 4 lots for brevity
    sub = df[df['SystemCodeNumber'] == lot]
    source = ColumnDataSource(sub)
    p = figure(title=f"Pricing for Lot {lot}", x_axis_type='datetime', width=400, height=250)
    p.line('Timestamp', 'Price', source=source, color='navy', legend_label='Demand-Based Price', line_width=2)
    p.line('Timestamp', 'PriceLinear', source=source, color='orange', legend_label='Linear Price', line_width=2, line_dash='dashed')
    p.add_tools(HoverTool(tooltips=[('Time', '@Timestamp{%F %T}'), ('Price', '@Price{0.2f}'), ('Linear', '@PriceLinear{0.2f}')], formatters={'@Timestamp': 'datetime'}))
    p.legend.location = 'top_left'
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    plots.append(p)

grid = gridplot(plots, ncols=2)
show(grid)

6. Competitor Price Comparison
To account for competitive dynamics, we compare each lot’s price with the average price of its three geographically closest competitors. This visualization helps justify pricing decisions in the context of local market conditions.

In [8]:
from scipy.spatial import distance_matrix

# Get coordinates for each lot
coords = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first()
dist_mat = pd.DataFrame(distance_matrix(coords.values, coords.values), index=coords.index, columns=coords.index)

# Find 3 nearest competitors for each lot
def get_competitors(lot):
    return dist_mat.loc[lot].sort_values()[1:4].index.tolist()

# Example: Compare first lot with its competitors
lot = df['SystemCodeNumber'].unique()[0]
competitors = get_competitors(lot)
sub = df[df['SystemCodeNumber'] == lot].copy()
sub['AvgCompetitorPrice'] = 0

for idx, row in sub.iterrows():
    ts = row['Timestamp']
    comp_prices = []
    for comp in competitors:
        comp_row = df[(df['SystemCodeNumber'] == comp) & (df['Timestamp'] == ts)]
        if not comp_row.empty:
            comp_prices.append(comp_row['Price'].values[0])
    if comp_prices:
        sub.at[idx, 'AvgCompetitorPrice'] = np.mean(comp_prices)
    else:
        sub.at[idx, 'AvgCompetitorPrice'] = np.nan

source = ColumnDataSource(sub)
p = figure(title=f"Lot {lot} vs Competitors", x_axis_type='datetime', width=600, height=300)
p.line('Timestamp', 'Price', source=source, color='navy', legend_label='Lot Price', line_width=2)
p.line('Timestamp', 'AvgCompetitorPrice', source=source, color='red', legend_label='Avg Competitor Price', line_width=2, line_dash='dashed')
p.add_tools(HoverTool(tooltips=[('Time', '@Timestamp{%F %T}'), ('Price', '@Price{0.2f}'), ('Competitor', '@AvgCompetitorPrice{0.2f}')], formatters={'@Timestamp': 'datetime'}))
p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price ($)'
show(p)

/tmp/ipython-input-8-3286800485.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.976338488694259' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub.at[idx, 'AvgCompetitorPrice'] = np.mean(comp_prices)


7. Demand Function Visualization
This plot shows how the normalized demand score changes over time for a selected parking lot. It provides insight into the underlying drivers of price changes and helps justify the model’s responsiveness to real-world factors.

In [9]:
lot = df['SystemCodeNumber'].unique()[0]
sub = df[df['SystemCodeNumber'] == lot]
source = ColumnDataSource(sub)
p = figure(title=f"Normalized Demand for Lot {lot}", x_axis_type='datetime', width=600, height=300)
p.line('Timestamp', 'DemandNorm', source=source, color='green', legend_label='Normalized Demand', line_width=2)
p.add_tools(HoverTool(tooltips=[('Time', '@Timestamp{%F %T}'), ('Demand', '@DemandNorm{0.2f}')], formatters={'@Timestamp': 'datetime'}))
p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Normalized Demand'
show(p)

8. Queue Length and Occupancy Over Time
This visualization tracks both the queue length and occupancy rate for a parking lot over time. It contextualizes demand and pricing, highlighting periods of high congestion or underutilization.

In [10]:
from bokeh.models import LinearAxis, Range1d

lot = df['SystemCodeNumber'].unique()[0]
sub = df[df['SystemCodeNumber'] == lot]
source = ColumnDataSource(sub)
p = figure(title=f"Queue Length & Occupancy for Lot {lot}", x_axis_type='datetime', width=600, height=300)
p.line('Timestamp', 'QueueLength', source=source, color='purple', legend_label='Queue Length', line_width=2)
p.extra_y_ranges = {"occ": Range1d(start=0, end=1)}
p.add_layout(LinearAxis(y_range_name="occ", axis_label="Occupancy Rate"), 'right')
p.line('Timestamp', 'OccupancyRate', source=source, color='orange', legend_label='Occupancy Rate', line_width=2, y_range_name="occ")
p.add_tools(HoverTool(tooltips=[('Time', '@Timestamp{%F %T}'), ('Queue', '@QueueLength'), ('Occupancy', '@OccupancyRate{0.2f}')], formatters={'@Timestamp': 'datetime'}))
p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Queue Length'
show(p)